In [1]:
import pandas as pd
import re
from unidecode import unidecode
from collections import defaultdict

In [2]:
# Ruta del archivo (ajusta si es necesario)
ruta_archivo = "DatosTecnicaTurboshop.xlsx"

# Cargar el archivo Excel y ver los nombres de las hojas
xls = pd.ExcelFile(ruta_archivo)
print("Hojas disponibles:", xls.sheet_names)


Hojas disponibles: ['F1', 'F2', 'F3', 'F4']


In [3]:
# Cargar todas las hojas en un diccionario
hojas = {nombre: xls.parse(nombre) for nombre in xls.sheet_names}

# Mostrar las primeras filas de cada hoja
for nombre, df in hojas.items():
    print(f"\n--- {nombre} ---")
    display(df.head())


--- F1 ---


,Codigo,Nombre,Aplicaciones
0,T181-1745-2,"Cadena de Distribucion, larga, doble, 94 eslab...",NaN
1,T181-1746-0,Cadena Larga Distribucion Toyota Corolla 1600C...,NaN
2,T181-2826-8,"Cadena de Distribucion, Superior, 84 eslabones...",NaN
3,T181-2825-K,Cadena Distribucion Chevrolet N300,NaN
4,T181-3979-0,"Cadena de Distribucion, 90 Eslabones, simple",NaN



--- F2 ---


,Codigo,Nombre,Aplicaciones
0,9075788,CALIPER DELANTERO IZQUIERDO (NORTARGE),SAIL 1.4 2011/2018
1,9075789,CALIPER DELANTERO DERECHO (NORTARGE),SAIL 1.4 2011/2018
2,9075789,CALIPER DELANTERO DERECHO (UXCLENT),SAIL 1.4 2011/2018
3,9075910,FILTRO POLEN (KENDALL),SAIL 1.4 1.5 2010/2021
4,9114603,SENSOR VELOCIDAD (NETMOTORS),CORSA 1.4 1.6 1993/1998-ASTRA 1.4 1.6 1992/199...



--- F3 ---


,Codigo,Nombre,Aplicaciones
0,2996724-5,RADIADOR MT L300 2.4 92-98+2.0-2.5 96-11 T/M C...,NaN
1,041010021,ACEITE 75W90 OPTIMUS DIF+CAJA T/M 1LT SEMISIN 648,NaN
2,OK7AA-52-910,PTA FRONTAL LH 2005> GENUINO,NaN
3,280085,PRENSA EMB HY SON EF VALEO,NaN
4,71911-43900,PTA FRONTAL 98-2004 LH VAN,NaN



--- F4 ---


,PATENTE,PATENTE_DV,TIPO,MARCA,MODELO,AÑO_FAB,COLOR
0,LTXZ55,4,AUTOMOVIL,HYUNDAI,GRAND I10 BA PE HB 1,2020,PLATEADO
1,LTXZ80,5,CAMIONETA,SSANGYONG,MUSSO GRAND 4X4 2.2,2020,GRIS
2,LTXZ83,K,STATION WAGON,FORD,EXPLORER XLT 3.5 AT,2020,BLANCO
3,LTYB45,1,AUTOMOVIL,KIA,SOLUTO LX 1.4,2020,PLATEADO
4,LTYB83,4,CAMIONETA,RENAULT,OROCH PICK UP 1.6,2020,BLANCO


In [4]:
# 1. Cargar los datos
df = pd.read_excel("DatosTecnicaTurboshop.xlsx", sheet_name="F4")
df['MODELO'] = df['MODELO'].astype(str).str.upper().str.strip()

In [5]:
# 2. Función mejorada para extraer tipo de vehículo (con énfasis en HATCHBACK)
def extraer_tipo_vehiculo(texto):
    # Ordenamos los patrones de más específico a menos específico
    patrones_tipo = [
        (r'\bHATCH\s?BACK\b', 'HATCHBACK'),  # Primero este para captar "HATCH BACK"
        (r'\bSTATION\sWAGON\b', 'SW'),
        (r'\bPICK\s?UP\b', 'PICKUP'),
        (r'\bMINIBUS\b', 'MINIBUS'),
        (r'\bSUV\b', 'SUV'),
        (r'\bSEDAN\b', 'SEDAN'),
        (r'\bCOUPE\b', 'COUPE'),
        (r'\bHB\b', 'HATCHBACK'),  # HB como abreviatura
        (r'\b5P\b', '5P'),      # 3 puertas
        (r'\b3P\b', 'HATCHBACK'),      # 3 puertas
        (r'\bHATCH\b', 'HATCHBACK') # Solo "HATCH"
    ]
    
    for patron, tipo in patrones_tipo:
        if re.search(patron, texto, flags=re.IGNORECASE):
            # Reemplazamos todas las ocurrencias
            texto = re.sub(patron, '', texto, flags=re.IGNORECASE).strip()
            return tipo, texto
    return None, texto

In [6]:
# 3. Función de extracción completa
def extraer_detalles_modelo(texto):
    detalles = {
        'texto_original': texto,
        'cilindrada': None,
        'traccion': None,
        'transmision': None,
        'tipo_vehiculo': None,
        'modelo_base': None,
        'version': None
    }
    
    if not isinstance(texto, str) or texto == 'NAN':
        return detalles
    
    texto_limpio = texto
    
    # Primero extraer el tipo de vehículo (ahora con prioridad)
    tipo, texto_limpio = extraer_tipo_vehiculo(texto_limpio)
    detalles['tipo_vehiculo'] = tipo
    
    # Luego extraer otros componentes técnicos
    cilindrada_match = re.search(r'\b(\d\.\d)\b', texto_limpio)
    if cilindrada_match:
        detalles['cilindrada'] = cilindrada_match.group(1)
        texto_limpio = texto_limpio.replace(detalles['cilindrada'], '')
    
    traccion_match = re.search(r'\b([24][Xx][24])\b', texto_limpio)
    if traccion_match:
        detalles['traccion'] = traccion_match.group(1).upper()
        texto_limpio = texto_limpio.replace(traccion_match.group(1), '')
    
    transmision_match = re.search(r'\b(AUT|AT|MT|CVT)\b', texto_limpio)
    if transmision_match:
        detalles['transmision'] = transmision_match.group(1)
        texto_limpio = texto_limpio.replace(detalles['transmision'], '')
    
    # Limpieza final
    texto_limpio = re.sub(r'[^A-Z0-9 ]', '', texto_limpio)
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio).strip()
    
    # Extracción de modelo base y versión según reglas especificadas
    palabras = texto_limpio.split()
    
    if len(palabras) > 1:
        detalles['modelo_base'] = palabras[0]
        detalles['version'] = ' '.join(palabras[1:])
    elif len(palabras) == 1:
        detalles['modelo_base'] = palabras[0]
        detalles['version'] = ''
    else:
        detalles['modelo_base'] = ''
        detalles['version'] = ''

    
    return detalles


In [7]:
# 3. Aplicar la función y crear nuevas columnas
detalles = pd.DataFrame(df['MODELO'].apply(extraer_detalles_modelo).tolist())
df = pd.concat([df, detalles], axis=1)

In [8]:
df

,PATENTE,PATENTE_DV,TIPO,MARCA,MODELO,AÑO_FAB,COLOR,texto_original,cilindrada,traccion,transmision,tipo_vehiculo,modelo_base,version
0,LTXZ55,4,AUTOMOVIL,HYUNDAI,GRAND I10 BA PE HB 1,2020,PLATEADO,GRAND I10 BA PE HB 1,None,None,None,HATCHBACK,GRAND,I10 BA PE 1
1,LTXZ80,5,CAMIONETA,SSANGYONG,MUSSO GRAND 4X4 2.2,2020,GRIS,MUSSO GRAND 4X4 2.2,2.2,4X4,None,None,MUSSO,GRAND
2,LTXZ83,K,STATION WAGON,FORD,EXPLORER XLT 3.5 AT,2020,BLANCO,EXPLORER XLT 3.5 AT,3.5,None,AT,None,EXPLORER,XLT
3,LTYB45,1,AUTOMOVIL,KIA,SOLUTO LX 1.4,2020,PLATEADO,SOLUTO LX 1.4,1.4,None,None,None,SOLUTO,LX
4,LTYB83,4,CAMIONETA,RENAULT,OROCH PICK UP 1.6,2020,BLANCO,OROCH PICK UP 1.6,1.6,None,None,PICKUP,OROCH,
5,LTYB93,1,CAMIONETA,MITSUBISHI,L200 WORK CR WF 2.4,2020,GRIS,L200 WORK CR WF 2.4,2.4,None,None,None,L200,WORK CR WF
6,LTYC15,6,STATION WAGON,HYUNDAI,CRETA GS PE 1.6 AUT,2020,GRIS,CRETA GS PE 1.6 AUT,1.6,None,AUT,None,CRETA,GS PE
7,LTYC19,9,STATION WAGON,MG,ZS 5P 1.5,2020,BLANCO,ZS 5P 1.5,1.5,None,None,5P,ZS,
8,LTYC39,3,STATION WAGON,SSANGYONG,TIVOLI 1.6 AUT,2019,PLATEADO,TIVOLI 1.6 AUT,1.6,None,AUT,None,TIVOLI,
9,LTYC41,5,AUTOMOVIL,CHEVROLET,SAIL 1.5,2020,CAFE,SAIL 1.5,1.5,None,None,None,SAIL,


In [9]:
# Diccionario de correcciones completas (modelo y versión)
correcciones_completas = {
    # HYUNDAI GRAND I10
    ('HYUNDAI', 'GRAND'): {
        'modelo': 'GRAND I10',
        'version': lambda x: x.replace('I10', '').strip()
    },
    
    # NISSAN X-TRAIL
    ('NISSAN', 'X'): {
        'modelo': 'X-TRAIL',
        'version': lambda x: x.replace('TRAIL', '').strip()
    },
    
    # JAC NEW REFINE
    ('JAC', 'NEW'): {
        'modelo': 'NEW REFINE',
        'version': lambda x: x.replace('REFINE', '').strip()
    },
    
    # NISSAN NP300 (unificar con NAVARA)
    ('NISSAN', 'NP300'): {
        'modelo': 'NAVARA',
        'version': lambda x: x.replace('NAVARA', '').strip()
    },
    
    ('SSANGYONG', 'GRAND'): {
        'modelo': 'MUSSO',
        'version': lambda x: x.replace('MUSSO', 'GRAND').strip()
    },
    
    # HYUNDAI GRAND I10 (caso alternativo)
    ('HYUNDAI', 'GRAND I10'): {
        'modelo': 'GRAND I10',
        'version': lambda x: x.replace('I10', '').strip()
    }
}

# Función para aplicar correcciones completas
def corregir_modelo_y_version(row):
    marca = row['MARCA']
    modelo_actual = row['modelo_base']
    version_actual = row['version']
    
    # Verificar si hay corrección para esta marca y modelo
    if (marca, modelo_actual) in correcciones_completas:
        correccion = correcciones_completas[(marca, modelo_actual)]
        nuevo_modelo = correccion['modelo']
        nueva_version = correccion['version'](version_actual)
        return pd.Series([nuevo_modelo, nueva_version])
    
    # Caso especial para GRAND I10 que aparece de dos formas
    if marca == 'HYUNDAI' and modelo_actual == 'GRAND' and 'I10' in version_actual:
        nuevo_modelo = 'GRAND I10'
        nueva_version = version_actual.replace('I10', '').strip()
        return pd.Series([nuevo_modelo, nueva_version])
    
    return pd.Series([row['modelo_base'], row['version']])

# Aplicar correcciones
df[['modelo_base_corregido', 'version_corregida']] = df.apply(corregir_modelo_y_version, axis=1)

In [10]:
df_4 = df[['MARCA', 'modelo_base_corregido', 'version_corregida', 'cilindrada', 'AÑO_FAB']]
df_4

,MARCA,modelo_base_corregido,version_corregida,cilindrada,AÑO_FAB
0,HYUNDAI,GRAND I10,BA PE 1,None,2020
1,SSANGYONG,MUSSO,GRAND,2.2,2020
2,FORD,EXPLORER,XLT,3.5,2020
3,KIA,SOLUTO,LX,1.4,2020
4,RENAULT,OROCH,,1.6,2020
5,MITSUBISHI,L200,WORK CR WF,2.4,2020
6,HYUNDAI,CRETA,GS PE,1.6,2020
7,MG,ZS,,1.5,2020
8,SSANGYONG,TIVOLI,,1.6,2019
9,CHEVROLET,SAIL,,1.5,2020
